Середины интервалов, относительные частоты, условные варианты, произведения

In [92]:
import pandas as pd
import math


borders = []
for i in range(7):
    borders.append((0.495 + 0.5 * i))

mid_intervals = []
for i in range(6):
    mid_intervals.append((borders[i] + borders[i + 1])/2)

p_i = [0.0465, 0.2791, 0.2558, 0.2558, 0.1163, 0.0465] # из предыдущей работы

c = mid_intervals[3]
h = 0.5
u_i = []
ui_pi = []
ui2_pi = []
ui3_pi = []
ui4_pi = []
for i  in range(6):
    u_i.append((mid_intervals[i] - c)/h)
    ui_pi.append(u_i[i] * p_i[i])
    ui2_pi.append(pow(u_i[i],2) * p_i[i])
    ui3_pi.append(pow(u_i[i],3) * p_i[i])
    ui4_pi.append(pow(u_i[i],4) * p_i[i])

df = pd.DataFrame({'x_i': mid_intervals, 'p_i': p_i, 'u_i': u_i, 'ui_pi': ui_pi, 'ui2_pi': ui2_pi, 'ui3_pi': ui3_pi, 'ui4_pi': ui4_pi})
df = df.round(4)
print(df)

     x_i     p_i  u_i   ui_pi  ui2_pi  ui3_pi  ui4_pi
0  0.745  0.0465 -3.0 -0.1395  0.4185 -1.2555  3.7665
1  1.245  0.2791 -2.0 -0.5582  1.1164 -2.2328  4.4656
2  1.745  0.2558 -1.0 -0.2558  0.2558 -0.2558  0.2558
3  2.245  0.2558  0.0  0.0000  0.0000  0.0000  0.0000
4  2.745  0.1163  1.0  0.1163  0.1163  0.1163  0.1163
5  3.245  0.0465  2.0  0.0930  0.1860  0.3720  0.7440


Условные и центральные эмпирические моменты

In [93]:
v = [sum(df['ui_pi'])]
for i in range(3):
    v.append(sum(df['ui'+ str(i+2) + '_pi']))

m = []
x_mid = v[0] * h + c
for j in range(4):
        m.append(sum(pow((df['x_i'] - x_mid), j + 1) * df['p_i']))
df_moment = pd.DataFrame({'v': v, 'm': m})
df_moment.round(3)


,v,m
0,-0.744,0.000
1,2.093,0.385
2,-3.256,0.074
3,9.348,0.356


In [94]:
x_mid_st = sum(df['x_i'] * df['p_i'])
print('Сравнение способов подсчета дисперсии и выборочного среднего')
disp_st = sum(pow((df['x_i'] - x_mid_st), 2) * df['p_i'])
df_st_compare = pd.DataFrame({'x_mid_st': [x_mid_st], 'x_mid_cond': [x_mid], 'disp_st': [disp_st], 'disp_cond': df_moment['m'][1] })
df_st_compare.round(3)

Сравнение способов подсчета дисперсии и выборочного среднего


,x_mid_st,x_mid_cond,disp_st,disp_cond
0,1.873,1.873,0.385,0.385


Сравнение дисперсий

In [95]:
disp_correct = disp_st * 6 / 5

disp_compare = pd.DataFrame({'disp_correct': [disp_correct], 'sko_correct': [math.sqrt(disp_correct)], 'disp_st': [disp_st], 'sko_st': [math.sqrt(disp_st)] })
disp_compare.round(3)

,disp_correct,sko_correct,disp_st,sko_st
0,0.462,0.68,0.385,0.62


Оценка коэфф. асимметрии и эксцесса

In [96]:
a_s = df_moment['m'][2] / pow(disp_compare['sko_st'][0], 3)
e_k = df_moment['m'][3] / pow(disp_compare['sko_st'][0], 4) - 3

df_as_ek = pd.DataFrame({'a_s': [a_s], 'e_k': [e_k]})
print(df_as_ek.round(3))

print('\na_s > 0 => распределение скошено вправо, 0.25 < a_s < 0.5 => асимметрия незначительна')
print('e_k < 0 => наше распределение более низкое и пологое относительно нормального')

    a_s    e_k
0  0.31 -0.598

a_s > 0 => распределение скошено вправо, 0.25 < a_s < 0.5 => асимметрия незначительна
e_k < 0 => наше распределение более низкое и пологое относительно нормального


Мода и медиана

In [104]:
sum_p = [0.0465, 0.3256, 0.5814, 0.8372, 0.9535, 1.0] # из предыдущей работы
mi_sum = [2, 14, 25, 36, 41, 43] # из предыдущей работы
cent = 43 // 2
index_med = 0
for i in range(5):
    if mi_sum[i] < cent <= mi_sum[i+1]:
        index_med = i
        break
x_med = df['x_i'][index_med] - h/2
m_e = x_med + h / df['p_i'][index_med] * (0.5 - sum_p[index_med - 1])

id_mod = df['p_i'].idxmax()
x_mod = df['x_i'][id_mod] - h/2
m_o = x_mod + h * (df['p_i'][id_mod] - df['p_i'][id_mod - 1]) / (df['p_i'][id_mod] - df['p_i'][id_mod - 1] + (df['p_i'][id_mod] - df['p_i'][id_mod + 1] ))

df_med_mod = pd.DataFrame({'m_o': [m_o], 'm_e': [m_e]})
df_med_mod.round(3)

,m_o,m_e
0,1.449,1.807


Коэффициент вариации

In [112]:
v_var = disp_compare['sko_st'][0]/ x_mid * 100

print(round(v_var,3))
print('коэффициент вариации > 33, значит совокупность неоднородная')

33.121
коэффициент вариации > 33, значит совокупность неоднородная
